In [61]:
import configparser
import praw
import pandas as pd
from datetime import datetime, timedelta

In [2]:
config = configparser.ConfigParser()
config.read('praw.ini')

['praw.ini']

In [3]:
client_id = config['DEFAULT']['client_id']
client_secret = config['DEFAULT']['client_secret']
user_agent = config['DEFAULT']['user_agent']
username = config['DEFAULT']['username']
password = config['DEFAULT']['password']

In [4]:
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent,username=username, 
                     password=password)

In [103]:
# This is for extracting reddit posts 
row_list = []
idx = []
for submission in reddit.subreddit('investing').controversial(limit=10000):
    unix_time = submission.created_utc
    posted_time = (datetime.utcfromtimestamp(unix_time)-timedelta(hours=4)).strftime('%Y-%m-%d %H:%M:%S')
    if submission.author != 'MasterCookSwag':
        row_list.append({'title':submission.title, 'description':submission.selftext, 'upvote_count':submission.score, 
'comment_count':submission.num_comments})
        idx.append(posted_time)
df = pd.DataFrame(row_list,columns=['title', 'description','upvote_count','comment_count'],index=idx).sort_index()

In [99]:
# This is for extracting reddit comments
row_list = []
idx = []
for submission in reddit.subreddit('investing').hot(limit=100):
    if submission.author != 'MasterCookSwag':
        submission.comment_sort = 'controversial'
        comments = submission.comments.list()
        for comment in comments:
            if isinstance(comment, praw.models.reddit.more.MoreComments) :
                continue
            else:
                row_list.append({'description':comment.body, 'upvote_count':comment.score})
                unix_time = comment.created_utc
                posted_time = (datetime.utcfromtimestamp(unix_time)-timedelta(hours=4)).strftime('%Y-%m-%d %H:%M:%S')
                idx.append(posted_time)
df = pd.DataFrame(row_list,columns=['description','upvote_count'],index=idx).sort_index()

In [100]:
df

,description,upvote_count
2020-03-19 13:23:41,"**Hi, welcome to /r/investing. Please note tha...",1
2020-03-19 13:30:37,"Why not, at the very least, demand companies s...",189
2020-03-19 13:43:36,1. I wonder if this means it's actually a ser...,34
2020-03-19 13:52:46,In many cases that stock is taken out of circu...,26
2020-03-19 14:12:26,"That sounds like justice, but financially sort...",144
...,...,...
2020-03-21 22:20:09,"thank you for that, very well explained!",1
2020-03-21 22:21:06,He is buying DAL [https://www.barrons.com/arti...,1
2020-03-21 22:21:08,More weird than dystopic. \n\nThe fed has made...,1
2020-03-21 22:21:18,"We'll see 16k this coming week, it won't be th...",1


In [102]:
df.to_csv('investing_reddit_comments.csv')

In [104]:
df.to_csv('investing_reddit_posts.csv')